In [4]:
import librosa
import math
import scipy
import matplotlib.pyplot as plt
import wave
import numpy
import sys
from scipy.signal import lfilter, hamming
from flask import Flask, render_template, Response, send_file
from scipy.io import wavfile
import matplotlib.pyplot as plt
import io
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from os import path
from time import sleep


In [5]:
app = Flask(__name__)

@app.route('/objective')
def Objectives():
    return render_template('Objective.html')

@app.route('/procedure')
def Procedure():
    return render_template('Procedure.html')


@app.route('/experiment')
def Experiment():
    return render_template('Experiment.html')


@app.route('/observations')
def Observations():
    return render_template('Observations.html')


@app.route('/references')
def References():
    return render_template('References.html')


@app.route('/feedback')
def Feedback():
    return render_template('Feedback.html')

@app.route('/tutorial')
def Tutorial():
    return render_template('Tutorial.html')

@app.route('/assessment')
def Assesment():
    return render_template('Assessment.html')

@app.route('/windowed/<file>/<type>')
def windowed_waveform(type, file):
    sleep(2)
    if(path.exists('static/images/windowed-'+type+'-wav'+file+'.png') is False) :
        fig = create_window_plot(type, file)

    return send_file('static/images/windowed-'+type+'-wav'+file+'.png', mimetype='image/gif')

@app.route('/stft/<file>/<nfft>')
def log_spectrum(file,nfft):
    sleep(2)
    if(path.exists('static/images/stft-wav'+file+'-nfft'+nfft+'.png') is False):
        fig = create_stft(file,nfft)

    return send_file('static/images/stft-wav'+file+'-nfft'+nfft+'.png', mimetype='image/gif')




In [6]:
audioFile = "ex1"

In [24]:
def createSpectAutoCorr(audioFile):
    y, sr = librosa.load("wav/"+audioFile+".wav")
    plt.plot(y)
    plt.savefig('static/images/'+audioFile+'.png')
    plt.close()
    autoCorr = librosa.autocorrelate(y)
    plt.yscale("log")
    plt.plot(autoCorr)
    plt.savefig('static/images/'+audioFile+'AutoCorr.png')
    plt.close()
    lpc= librosa.lpc(y,10)
    plt.plot(lpc)
    plt.savefig('static/images/'+audioFile+'lpc.png')
    plt.close()
    spf = wave.open("wav/"+audioFile+".wav", 'r') 
    x = spf.readframes(-1)
    x = numpy.fromstring(x, 'Int16')

    N = len(x)
    w = numpy.hamming(N)

    x1 = x * w
    x1 = lfilter([1], [1., 0.63], x1)

    rts = numpy.roots(lpc)
    rts = [r for r in rts if numpy.imag(r) >= 0]

    angz = numpy.arctan2(numpy.imag(rts), numpy.real(rts))

    Fs = spf.getframerate()
    frqs = sorted(angz * (Fs / (2 * math.pi)))
    fmin = 5
    fmax= 20000
    i_min = sr/fmax
    i_max = sr/fmin
    autoCorr[:int(i_min)] = 0
    autoCorr[int(i_max):] = 0
    i = autoCorr.argmax()
    f0 = float(sr)/i
    freqFile=open('freqFile.txt', "a")
    freqFile.write(str(frqs)+" "+str(f0)+"\n")
    freqFile.close()
    return frqs, f0

/home/shubhangi/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
/home/shubhangi/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


([428.88937, 1437.3114, 2220.111, 3866.1294, 5144.9336], 22050.0)

In [27]:
for i in range(1,5):
    createSpectAutoCorr('ex'+str(i))

/home/shubhangi/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
/home/shubhangi/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


In [1]:

if __name__ == '__main__':
    type = ['rectangular','hamming','hann','cosine']

    for file in range(1,3):
        for option in type:
            create_window_plot(option,file)

    nfft_values = [64,128,256,512,1024,2048, 4096,8192]

    for file in range(1,3):
        for nfft in nfft_values:
            create_stft(file, nfft)

    app.run()

NameError: name 'create_window_plot' is not defined